In [ ]:
import torch
torch.set_default_dtype(torch.float64)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tdg

To take an infinite volume limit of the free theory ($\tilde{a}=0$) requires thinking.  Recall that $ \tilde{\beta} = \frac{\beta}{ML^2}$ and we want to adjust L at constant $\beta$.

The issue is that we have no dimensionful way to fix L.  However, we can *scale* L in the following way.

Fix a particular starting nx0 and $\tilde{\beta}_0$.  Then as we change nx, if we kept the lattice spacing the same we would need to scale L by (nx/nx0).  Therefore $\tilde{\beta} = \tilde{\beta}_0 (nx0 / nx)^2$.

We can make a similar scaling for $\tilde{\mu} = \mu ML^2 $ which should go to $\tilde{\mu} = \tilde{\mu}_0 (nx/nx0)^2$.  But, we'll just set $\mu=0$ and then scaling 0 leaves it fixed.  The same thing happens for $\tilde{h}$.

In [ ]:
nx0 = 7
beta0 = 3./49
mu0 = 0.
h0 = torch.tensor([0.,0.,0.])

In [ ]:
NX = (7, 9, 11, 13, 15, 17, 19, 21, 23, 25,)

In [ ]:
ensembles = set()

In [ ]:
for nx in NX:
    L = tdg.Lattice(nx)
    spacetime = tdg.Spacetime(2, L)
    
    beta = beta0 * (nx0 / nx)**2
    mu   = mu0   * (nx / nx0)**2
    h    = h0    * (nx / nx0)**2
    
    S = tdg.Action(
        spacetime,
        tdg.Potential(-0.000000000000000001*tdg.LegoSphere([0,0])),
        beta=beta,
        mu=mu,
        h=h,
        )
    
    zero_configuration = spacetime.vector()
    cfgs = torch.stack((zero_configuration,))
    
    free = tdg.ensemble.GrandCanonical(S).from_configurations(cfgs).bootstrapped(1)
    ensembles.add(free)

In [ ]:
fig, ax = plt.subplots(10,1, sharex='col', figsize=(8,30))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx
    k2 = L.linearize(L.ksq)

    ax[0].plot((1/nx**2,), (e.N[0].real/nx**2,        ), label=f'{nx}', marker='o')
    ax[1].plot((1/nx**2,), (e.T_by_TF[0].real,  ), label=f'{nx}', marker='o')

    ax[2].plot((1/nx**2,), (e.kinetic_by_kF4[0].real,        ), label=f'{nx}', marker='o')
    ax[3].plot((1/nx**2,), (e.freeEnergy_by_kF4[0].real,  ), label=f'{nx}', marker='o')
    
    ax[4].plot((1/nx**2,), (e.w0_by_kF4[0].real, ), label=f'{nx}', marker='o')
    ax[5].plot((1/nx**2,), (e.w2_by_kF2[0].real, ), label=f'{nx}', marker='o')
    ax[6].plot((1/nx**2,), (e.w4[0].real,        ), label=f'{nx}', marker='o')
    
    ax[-3].plot((1/nx**2,), (e.b2_by_kF4[0].real,), label=f'{nx}', marker='o')
    ax[-2].plot((1/nx**2,), (e.b4_by_kF2[0].real,), label=f'{nx}', marker='o')
    ax[-1].plot((1/nx**2,), (e.b6       [0].real,), label=f'{nx}', marker='o')
    
ax[0].set_ylabel('N / nx^2')
ax[1].set_ylabel('T/TF')

ax[2].set_ylabel('k / kF^4')
ax[3].set_ylabel('f / kF^4')

ax[4].set_ylabel('w0 / kF^4')
ax[5].set_ylabel('w2 / kF^2')
ax[6].set_ylabel('w4')

ax[-3].set_ylabel('b2 / kF^4')
ax[-2].set_ylabel('b4 / kF^2')
ax[-1].set_ylabel('b6')


ax[-1].set_xlim([0, 0.022])
ax[-1].set_xlabel('1/nx^2')